In [21]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler
from scipy import stats
import matplotlib.pyplot as plt

In [22]:
real_data = pd.read_csv("C:/Users/Luke/OneDrive/Research/sg_data/sg_hole_14thru19_data_cavg.csv").drop(['Unnamed: 0','Unnamed: 0.1'], axis = 1)


In [23]:
real_data.head()

,player,year,plyr_year,course_name,round,c_idx,p_y_idx,y_idx,yr_ind_14,yr_ind_15,...,SG_drive_raw,SG_putt_raw,SG_appr_cavg,SG_short_cavg,SG_drive_cavg,SG_putt_cavg,SG_drive,SG_appr,SG_short,SG_putt
0,A.J. McInerney,2018,"('A.J. McInerney', 2018)",Bay Hill Club & Lodge,1,2,0,4,0,0,...,0.326278,-4.272,-0.468932,0.045342,-0.122122,0.074978,0.448400,-0.037484,1.114797,-4.346978
1,Aaron Wise,2018,"('Aaron Wise', 2018)",Bay Hill Club & Lodge,1,2,12,4,0,0,...,-0.091278,0.044,-0.468932,0.045342,-0.122122,0.074978,0.030844,4.383738,-0.045676,-0.030978
2,Adam Scott,2018,"('Adam Scott', 2018)",Bay Hill Club & Lodge,1,2,38,4,0,0,...,1.091778,1.327,-0.468932,0.045342,-0.122122,0.074978,1.213900,-0.799401,-0.802342,1.252022
3,Alex Cejka,2018,"('Alex Cejka', 2018)",Bay Hill Club & Lodge,1,2,62,4,0,0,...,-0.403889,-1.113,-0.468932,0.045342,-0.122122,0.074978,-0.281767,-1.678290,2.326769,-1.187978
4,Alex Noren,2018,"('Alex Noren', 2018)",Bay Hill Club & Lodge,1,2,69,4,0,0,...,0.224222,2.081,-0.468932,0.045342,-0.122122,0.074978,0.346344,-1.244595,-1.200842,2.006022


In [4]:
#to create first time but then use true values to change se
np.random.seed(42)
players = len(pd.unique(real_data['player']))
x_d = np.random.normal(0, 1, size = players) # players "true value" avg. standardized to tour average drive
x_d = stats.zscore(x_d)
x_a = np.random.normal(0, 1, size = players) # players "true value" avg. standardized to tour average drive
x_a = stats.zscore(x_a)
x_s = np.random.normal(0, 1, size = players) # players "true value" avg. standardized to tour average drive
x_s = stats.zscore(x_s)
x_p = np.random.normal(0, 1, size = players) # players "true value" avg. standardized to tour average drive
x_p = stats.zscore(x_p)
player_df = pd.DataFrame({'player': pd.unique(real_data['player']), 'p_sg_drive': x_d, 'p_sg_appr': x_a,'p_sg_short': x_s, 'p_sg_putt': x_p})

player_df.to_csv('C:/Users/Luke/OneDrive/Research/model_data/hole_SYN_pDF_09112022.csv')

In [7]:
courses = len(pd.unique(real_data['course_name']))


par_3s = np.random.choice([0, 1], size=courses, p=[.2, .8])
h_d = np.random.normal(0, 0.8, size = courses) 
h_d = stats.zscore(h_d)*par_3s
h_a = np.random.normal(0, 0.8, size = courses) 
h_a = stats.zscore(h_a)
h_s = np.random.normal(0, 0.6, size = courses) 
h_s = stats.zscore(h_s)
h_p = np.random.normal(0, 0.5, size = courses) 
h_p = stats.zscore(h_p)
h_d_d = np.random.normal(0.5, 0.12, size = courses)*par_3s
h_d_a = np.random.normal(0.5, 0.12, size = courses) 
h_d_s = np.random.normal(0.5, 0.12, size = courses) 
h_d_p = np.random.normal(0.5, 0.12, size = courses) 

hole_df = pd.DataFrame({'course_name': pd.unique(real_data['course_name']), 'h_sg_drive': h_d, 'h_sg_appr': h_a,'h_sg_short': h_s, 'h_sg_putt': h_p,\
                      'h_dsr_drive': h_d_d, 'h_dsr_appr': h_d_a,'h_dsr_short': h_d_s, 'h_dsr_putt': h_d_p,})
hole_df.to_csv('C:/Users/Luke/OneDrive/Research/model_data/hole_SYN_cDF_09112022.csv')

In [15]:
player_df = pd.read_csv('C:/Users/Luke/OneDrive/Research/model_data/hole_SYN_pDF_09112022.csv')
hole_df = pd.read_csv('C:/Users/Luke/OneDrive/Research/model_data/hole_SYN_cDF_09112022.csv')

print('data shape: ', real_data.shape)
syn_data__ = pd.merge(real_data, player_df, how = 'left', on = 'player')
print('after player join: ', syn_data__.shape)
syn_data = pd.merge(syn_data__, hole_df, how = 'left', on = 'course_name').drop(['SG_appr','SG_short','SG_putt','SG_drive'], axis=1)
print('after course join: ', syn_data.shape)

par3 = [1 if i != 0 else 0 for i in syn_data['h_sg_drive']]


syn_data['p_sg_drive'] = stats.zscore(syn_data['p_sg_drive'])
syn_data['p_sg_appr'] = stats.zscore(syn_data['p_sg_appr'])
syn_data['p_sg_short'] = stats.zscore(syn_data['p_sg_short'])
syn_data['p_sg_putt'] = stats.zscore(syn_data['p_sg_putt'])

syn_data['h_sg_drive'] = stats.zscore(syn_data['h_sg_drive'])
syn_data['h_sg_drive'] = syn_data['h_sg_drive']*par3
syn_data['h_sg_appr'] = stats.zscore(syn_data['h_sg_appr'])
syn_data['h_sg_short'] = stats.zscore(syn_data['h_sg_short'])
syn_data['h_sg_putt'] = stats.zscore(syn_data['h_sg_putt'])

w_p_df = syn_data[['player','p_y_idx','p_sg_drive','p_sg_appr','p_sg_short','p_sg_putt']].drop_duplicates().reset_index(drop=True)
w_p_df.columns = ['player','p_y_idx','sg_drive','sg_appr','sg_short','sg_putt']
w_p_df = w_p_df.sort_values(by='p_y_idx').reset_index(drop=True)
w_p_df.to_csv('C:/Users/Luke/OneDrive/Research/model_data/hole_wSYN_pDF_09112022.csv')

w_h_df = syn_data[['course_name','c_idx','h_sg_drive','h_sg_appr','h_sg_short','h_sg_putt', 'h_dsr_drive','h_dsr_appr','h_dsr_short','h_dsr_putt']].drop_duplicates().reset_index(drop=True)
w_h_df.columns = ['course_name','c_idx','h_sg_drive','h_sg_appr','h_sg_short','h_sg_putt', 'h_dsr_drive','h_dsr_appr','h_dsr_short','h_dsr_putt']
w_h_df = w_h_df.sort_values(by='c_idx').reset_index(drop=True)
w_h_df.to_csv('C:/Users/Luke/OneDrive/Research/model_data/hole_wSYN_cDF_09112022.csv')


data shape:  (83621, 26)
after player join:  (83621, 31)
after course join:  (83621, 36)


In [20]:
se = 0.5


syn_data['SG_drive'] = syn_data['p_sg_drive']*syn_data['h_dsr_drive'] - syn_data['h_sg_drive'] + np.random.normal(0, se, size = len(syn_data['player']))
syn_data['SG_appr'] = syn_data['p_sg_appr']*syn_data['h_dsr_appr'] - syn_data['h_sg_appr'] + np.random.normal(0, se, size = len(syn_data['player']))
syn_data['SG_short'] = syn_data['p_sg_short']*syn_data['h_dsr_short'] - syn_data['h_sg_short'] + np.random.normal(0, se, size = len(syn_data['player']))
syn_data['SG_putt'] = syn_data['p_sg_putt']*syn_data['h_dsr_putt'] - syn_data['h_sg_putt'] + np.random.normal(0, se, size = len(syn_data['player']))

syn_data['SG_drive'] = [syn_data['SG_drive'][i] if val != 0 else 0 for i,val in enumerate(par3)]
#syn_data_final = syn_data.drop(['p_sg_drive','p_sg_appr','p_sg_short','p_sg_putt','c_sg_drive','c_sg_appr','c_sg_short','c_sg_putt','c_dsr_drive','c_dsr_appr','c_dsr_short','c_dsr_putt'], axis = 1)
syn_data_final = syn_data



syn_data_final.to_csv('C:/Users/Luke/OneDrive/Research/model_data/hole_syn05_realdata_09112022.csv')